In [ ]:
!pip3 install -U llama-index llama-index-embeddings-dashscope llama-index-llms-openai-like llama-index-llms-dashscope

1. 解析文件，创建索引和提问引擎

只包括创建索引、创建提问引擎、提问和生成解答 4 个步骤。

In [ ]:
# 导入依赖
import os
from llama_index.embeddings.dashscope import DashScopeEmbedding, DashScopeTextEmbeddingModels
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai_like import OpenAILike
# 这两行代码是用于消除 WARNING 警告信息，避免干扰阅读学习，生产环境中建议根据需要来设置日志级别
import logging
logging.basicConfig(level=logging.ERROR)

print("正在解析文件...")
# LlamaIndex提供了SimpleDirectoryReader方法，可以直接将指定文件夹中的文件加载为document对象，对应着解析过程
documents = SimpleDirectoryReader('./docs').load_data()

print("正在创建索引...")
# from_documents方法包含切片与建立索引步骤
index = VectorStoreIndex.from_documents(
  documents,
  # 指定embedding 模型
  embed_model=DashScopeEmbedding(
    # 你也可以使用阿里云提供的其它embedding模型：https://help.aliyun.com/zh/model-studio/getting-started/models#3383780daf8hw
    model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
  )
)

print("正在创建提问引擎...")
query_engine = index.as_query_engine(
  # 设置为流式输出
  streaming=True,
  # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
  llm=OpenAILike(
    model="qwen-plus",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    is_chat_model=True
  )
)

print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
# 采用流式输出
streaming_response.print_response_stream()

2. 保存和加载索引

In [ ]:
# 将索引保存为本地文件
# 创建索引消耗的时间比较长。如果能够将索引保存到本地，并在需要使用的时候直接加载，
# 而不是重新建立索引，那就可以大幅提升回复的速度
index.storage_context.persist("knowledge_base/test")
print("索引文件保存到了knowledge_base/test")

In [ ]:
# 将本地索引文件加载为索引
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="knowledge_base/test")
index = load_index_from_storage(storage_context, embed_model=DashScopeEmbedding(
  model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
))
print("成功从knowledge_base/test路径加载索引")

3. 从本地加载索引后，可以再次进行提问，测试是否可以进行正常工作

In [ ]:
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
  # 设置为流式输出
  streaming=True,
  # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
  llm=OpenAILike(
    model="qwen-plus",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    is_chat_model=True
  )
)

print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
streaming_response.print_response_stream()


4. 封装为自定义函数库

In [ ]:
from chatbot import rag

# 引文在前面的步骤中已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
index = rag.load_index()
query_engine = rag.create_query_engine(index=index)

rag.ask('我们公司项目管理应该用什么工具', query_engine=query_engine)

5. 多轮对话 RAG

In [ ]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.llms.openai_like import OpenAILike
import os

custom_prompt = PromptTemplate(
  """\
  给定一段对话（在人类与助手之间）以及来自人类的后续信息，
  请将该信息改写为一个独立的问题，并在其中包含对话中的所有相关上下文。
  <聊天记录>
  {chat_history}
  <后续信息>
  {question}
  <独立的问题>
  """
)

# 历史对话信息
custom_chat_history = [
  ChatMessage(role=MessageRole.USER, content="内容开发工程师细分类型是什么?",),
  ChatMessage(role=MessageRole.ASSISTANT, content="综合技术岗位."),
]

query_engine = index.as_query_engine(
  # 设置为流式输出
  streaming = True,
  # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
  llm = OpenAILike(
    model = "qwen-plus",
    api_base = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key = os.getenv("DASHSCOPE_API_KEY"),
    is_chat_model = True
  )
)

chat_engine = CondenseQuestionChatEngine.from_defaults(
  query_engine = query_engine,
  condense_question_prompt = custom_prompt,
  chat_history = custom_chat_history,
  llm = OpenAILike(
    model = "qwen-plus",
    api_base = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key = os.getenv("DASHSCOPE_API_KEY"),
    is_chat_model = True
  ),
  verbose = True
)

streaming_response = chat_engine.stream_chat("核心职责是什么")
for token in streaming_response.response_gen:
  print(token, end="")


Querying with: 内容开发工程师（综合技术岗位）的核心职责是什么？
内容开发工程师（综合技术岗位）的核心职责包括：

- **内容创作与编辑**：根据项目需求，撰写和编辑各类内容，确保信息准确且符合目标受众的需求。
- **内容管理**：使用相关工具和技术对内容进行分类、标签化、更新和维护，实现内容的高效管理。
- **内容优化**：通过SEO等手段提升内容的可见性与吸引力，增强用户参与度。
- **技术整合**：与开发团队协作，将内容无缝集成到网站、应用程序或其他数字平台中。
- **数据分析**：分析用户行为数据，评估内容效果，为优化内容策略提供数据支持。